In [24]:
import sys
sys.path.insert(0, '..')

import torch
import os
import wandb
import random
import numpy as np
import torch
from torch import nn
from torch.optim import Adam
from ignite.handlers.param_scheduler import create_lr_scheduler_with_warmup
from torch.utils.data import DataLoader
from datetime import datetime
import matplotlib.pyplot as plt
import pandas as pd
from scipy import stats
from tqdm import tqdm
from matplotlib import cm
import seaborn as sns
import matplotlib.lines as mlines
from sklearn.decomposition import PCA
from openTSNE import TSNE
from PIL import Image
import umap
import torch.nn.functional as F
from scipy.spatial.distance import cdist
from torch.optim.lr_scheduler import ExponentialLR, ReduceLROnPlateau

from core.final.dataset import PSMDataset
from core.final.model import GalSpecNet, MetaModel, Informer, AstroModel
from core.final.trainer import Trainer
from core.final.loss import CLIPLoss

In [19]:
def set_random_seeds(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    np.random.seed(random_seed)
    random.seed(random_seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_random_seeds(42)

In [44]:
class Model2D(nn.Module):
    def __init__(self, encoder):
        super(Model2D, self).__init__()

        self.encoder = encoder
        
        for param in self.encoder.parameters():
            param.requires_grad = False

        self.p_fc = nn.Linear(encoder.photometry_proj.out_features, 2)
        self.s_fc = nn.Linear(encoder.spectra_proj.out_features, 2)
        self.m_fc = nn.Linear(encoder.metadata_proj.out_features, 2)

    def forward(self, photometry, photometry_mask, spectra, metadata):
        p_emb, s_emb, m_emb = self.encoder.get_embeddings(photometry, photometry_mask, spectra, metadata)
        
        p_emb = self.p_fc(p_emb)
        s_emb = self.s_fc(s_emb)
        m_emb = self.m_fc(m_emb)

        logits_ps = p_emb @ s_emb.T
        logits_sm = s_emb @ m_emb.T
        logits_mp = m_emb @ p_emb.T

        return logits_ps, logits_sm, logits_mp

In [46]:
run_id = 'MeriDK/AstroCLIPResults3/2wz4ysvn'
api = wandb.Api()
run = api.run(run_id)
config = run.config
config['use_wandb'] = False
config['save_weights'] = False

train_dataset = PSMDataset(config, split='train')
val_dataset = PSMDataset(config, split='val')

train_dataloader = DataLoader(train_dataset, batch_size=config['batch_size'], shuffle=True, drop_last=True,
                              num_workers=4)
val_dataloader = DataLoader(val_dataset, batch_size=config['batch_size'], shuffle=False)

device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

encoder = AstroModel(config)
encoder = encoder.to(device)

weights_path = os.path.join(config['weights_path'] + '-' + run_id.split('/')[-1], f'weights-best.pth')
encoder.load_state_dict(torch.load(weights_path, weights_only=False))

In [51]:
model = Model2D(encoder)
model = model.to(device)

optimizer = Adam(model.parameters(), lr=0.001)
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=config['factor'], patience=100)
criterion = CLIPLoss()

In [52]:
trainer = Trainer(model=model, optimizer=optimizer, scheduler=scheduler, warmup_scheduler=None,
                  criterion=criterion, device=device, config=config)

In [49]:
trainer.train_epoch(train_dataloader)

In [53]:
for _ in range(10):
    print(trainer.train_epoch(train_dataloader))

In [54]:
for _ in range(20):
    print(trainer.train_epoch(train_dataloader))

In [34]:
model.train()

total_loss = []
total_correct_predictions = 0
total_predictions = 0

for photometry, photometry_mask, spectra, metadata, labels in tqdm(train_dataloader):
    photometry, photometry_mask = photometry.to(device), photometry_mask.to(device)
    spectra, metadata = spectra.to(device), metadata.to(device)

    optimizer.zero_grad()
    logits_ps, logits_sm, logits_mp = model(photometry, photometry_mask, spectra, metadata)
    loss_ps, loss_sm, loss_mp = criterion(logits_ps, logits_sm, logits_mp)
    loss = loss_ps + loss_sm + loss_mp
    loss.backward()

    labels = torch.arange(logits_ps.shape[0], dtype=torch.int64, device=self.device)

    prob_ps = (F.softmax(logits_ps, dim=1) + F.softmax(logits_ps.transpose(-1, -2), dim=1)) / 2
    prob_sm = (F.softmax(logits_sm, dim=1) + F.softmax(logits_sm.transpose(-1, -2), dim=1)) / 2
    prob_mp = (F.softmax(logits_mp, dim=1) + F.softmax(logits_mp.transpose(-1, -2), dim=1)) / 2
    prob = (prob_ps + prob_sm + prob_mp) / 3

    _, pred_labels = torch.max(prob, dim=1)
    correct_predictions = (pred_labels == labels).sum().item()

    total_correct_predictions += correct_predictions
    total_predictions += labels.size(0)
    total_loss.append(loss.item())

In [35]:
photometry, photometry_mask, spectra, metadata

In [28]:
train_loss, train_acc = trainer.train_epoch(train_dataloader)

In [32]:
photometry, photometry_mask, spectra, metadata, labels = next(iter(train_dataloader))

with torch.no_grad():
    photometry, photometry_mask = photometry.to(device), photometry_mask.to(device)
    spectra, metadata = spectra.to(device), metadata.to(device)

    trainer.model(photometry, photometry_mask, spectra, metadata)